In [13]:
import pandas as pd
from pathlib import Path
from habanero import Crossref

In [ ]:
base_path = 'source/'
use_column = ['Article Title', 'Abstract', 'DOI']

In [ ]:
def excel_list():
    folder = Path(base_path)
    return [f.name for f in folder.iterdir() if f.is_file()]

#e = excel_list()

In [ ]:
def make_one(excel_list):

    concat_list = []

    def cutted_df(file_name):
        d = pd.read_excel(base_path + file_name)
        d = d[use_column]
        d = d.dropna(subset = use_column)
        return d
    
    for e in excel_list:
        concat_list.append(cutted_df(e))

    result = pd.concat(concat_list, axis=0, ignore_index=True)
    return result.drop_duplicates(subset=['DOI'])

#m = make_one(e)
#m.to_csv('make_one.csv', index=False)

In [ ]:
def add_date(doi):
    paper = Crossref().works(ids = doi)['message']
    for field in ['published-print', 'published-online', 'issued', 'created']:
        if field in paper:
            arr = paper[field]['date-parts'][0]
            if len(arr) < 2:
                return pd.Timestamp(year=arr[0], month=1, day=1)
            return pd.Timestamp(year=arr[0], month=arr[1], day=1)
    raise Exception("발행일 관련 정보가 없습니다.")

def doi_to_date(file_name):
    df = pd.read_csv(file_name)
    df['Date'] = df['DOI'].apply(add_date)
    df = df.drop(['DOI'])
    return df

doi_to_date('make_one.csv')